In [1]:
import os
import sys
import re
import requests
from bs4 import BeautifulSoup
import pickle
sys.path.append('../')
import confana
#from common import *

In [ ]:
# configurations
urlpost = '/Conference'
since = 2018
until = 2025

urlpre = 'https://openreview.net/group?id=ICLR.cc/'
param = confana.set_param(urlpre, 'ICLR', since, until, 1, 1)

#urlpre = 'https://openreview.net/group?id=robot-learning.org/CoRL/'
#param = confana.set_param(urlpre, 'CoRL', since, until, 1, 1)

#urlpre = 'https://openreview.net/group?id=NeurIPS.cc/'
#param = confana.set_param(urlpre, 'NeurIPS', since, until, 1, 1)

histall = {}

for ii in range(since,until+1,1):
    yr = str(ii)
    url = urlpre+yr+urlpost
    pklfl = param.cachedir + '/' + 'summary' + yr
    conf = param.conf_prefix + yr
    print(conf,'...')

    if os.path.exists(pklfl):
        with open(pklfl,'rb') as f:
            authorss, titles, numoral, numsl, numposter = pickle.load(f)
            print(f'loading {pklfl}...')
    else:
        pu = confana.parseUrl(param,yr)
        normal, authorss, titles, numoral, numsl, numposter = pu.parseOpenReview(url)
        if normal < 0: 
            print('parsing failed...')

    print(r'oral: {}, spotlight: {}, poster: {}, total: {}'.format(numoral, numsl, numposter,len(titles)))
    pAT = confana.parseATlist(titles, authorss)
    jpauthors, jptitles, numallauthors, numjpauthors, numpapers, numjppaper = pAT.selectJP()
    hist = pAT.makehist(jpauthors,yr)

    with open(pklfl, 'wb') as f:
        pickle.dump((authorss, titles, numoral, numsl, numposter), f)

    print(f'\t{numjpauthors} Japanese authors out of all {numallauthors} authors')
    print(f'\t{numjppaper} Japanese related papers out of all {numpapers} papers')
        
    #print(jpauthors)
    #for ii in range(0,len(jpauthors)):
    #    print(f'{jpauthors[ii]}\t{jptitles[ii]}')
    pAT = confana.parseATlist(titles, authorss)
    histall = pAT.mergehist(histall,hist,yr)

ph = confana.parseHist(param,histall)
ph.plotsorted()

In [ ]:
import pickle
import re

for year in range(2018,2025,1):
    print("----")
    print(year)
    with open(param.cachedir + '/' +   'summary' + str(year) , 'rb' ) as f:
        authorss, titles, numoral, numsl, numposter = pickle.load(f)

    numpaper = len(titles)
    pt = confana.parseTitle(titles)
    stwordhist = pt.wordfreq()
    for item in stwordhist:
        ratio = item[1]/numpaper
        if ratio > 0.005:
            #print("{0:}\t{1:}\t{2:.5f}".format(item[0],item[1],ratio))
            print("{0:}\t{1:.5f}".format(item[0],ratio))
            #print("{0:}: {1:}".format(item[0],item[1]))
